In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import re
import nltk
from nltk.tokenize import TweetTokenizer
import string
import unidecode

import warnings
warnings.filterwarnings('ignore')

stopwords = nltk.corpus.stopwords.words('english')

In [2]:
tweets = pd.read_csv("../data/train.csv")

In [3]:
def concatenate(x,char):
    words = ""
    for word in x:
        if word.startswith(char):
            words = words + word + " "
    return words

def count_vowels(x):
    return (x.count('a') + x.count('e') + x.count('i') + x.count('o') + x.count('u'))

def count_short_words(x):
    count = 0
    words = x.split(' ')
    for word in words:
        if 1 <= len(word) <= 3:
            count += 1
    return count

def count_stopwords(x):
    count = 0
    words = x.split(' ')
    for word in words:
        if word in stopwords:
            count += 1
    return count

In [4]:
tweets["special_chars_count"] =  tweets["text"]
tweets["special_chars_count"] =  tweets["special_chars_count"].str.lower()
tweets["special_chars_count"] = tweets["special_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
tweets["special_chars_count"] = tweets["special_chars_count"].str.strip()
tweets["special_chars_count"] = tweets["special_chars_count"].apply(lambda x: re.sub(' +','', x))
tweets["special_chars_count"] = tweets["special_chars_count"].apply(lambda x: re.sub(r'[0-9]','', x))
tweets["special_chars_count"] = tweets["special_chars_count"].str.len()

tweets["hashtags"] = tweets["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'#'))
tweets["labels"] = tweets["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'@'))
tweets["hashtags_count"] = tweets["hashtags"].str.split(' ').apply(lambda x: len(x))-1
tweets["labels_count"] = tweets["labels"].str.split(' ').apply(lambda x: len(x))-1

tweets["num_chars_count"] = tweets["text"]
tweets["num_chars_count"] =  tweets["num_chars_count"].str.lower()
tweets["num_chars_count"] = tweets["num_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
tweets["num_chars_count"] = tweets["num_chars_count"].apply(lambda x: re.sub(r'[^\w]','',x))
tweets["num_chars_count"] = tweets["num_chars_count"].str.strip()
tweets["num_chars_count"] = tweets["num_chars_count"].str.len()

In [5]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [6]:
tweets["clean_text"] = tweets["text"].apply(lambda x: cleaning_text(x))
tweets.head()

,id,keyword,location,text,target,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,1,#earthquake,,1,0,0,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,1,,,0,0,0,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,3,,,0,0,0,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,2,#wildfires,,1,0,5,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,2,#alaska #wildfires,,2,0,0,got sent photo ruby alaska smoke wildfires pou...


In [7]:
tweets["text"] = tweets["text"].str.lower()
tweets["text"] = tweets["text"].apply(lambda x: re.sub('(?P<url>https?://[^\s]+)', ' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'[^\w]', ' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'_', ' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'[0-9]',' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(' +',' ', x))
tweets["text"] = tweets["text"].apply(lambda x: unidecode.unidecode(x))
tweets["text"] = tweets["text"].str.strip()
tweets["text_length"] = tweets["text"].str.len()

tweets["vowels_count"] = tweets["text"].apply(lambda x: count_vowels(x))
tweets["short_words_count"] = tweets["text"].apply(lambda x: count_short_words(x))
tweets["stopwords_count"] = tweets["text"].apply(lambda x: count_stopwords(x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'\b\w{1}\b', '', x))
tweets["words_count"] = tweets["text"].str.split(' ').apply(lambda x: len(x))

In [8]:
tweets.rename(columns={"target":"target_label"}, inplace=True)
tweets.head()

,id,keyword,location,text,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,clean_text,text_length,vowels_count,short_words_count,stopwords_count,words_count
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,,1,0,0,deeds reason earthquake may allah forgive us,68,25,7,6,13
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,,,0,0,0,forest fire near la ronge sask canada,37,13,1,0,7
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,,,0,0,0,residents asked shelter place notified officer...,130,45,9,11,22
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,,1,0,5,people receive wildfires evacuation orders cal...,56,24,1,1,7
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,,2,0,0,got sent photo ruby alaska smoke wildfires pou...,85,25,3,7,16


In [9]:
#BOW
vectorizer = CountVectorizer(stop_words='english')
df_text = tweets["clean_text"]
X = vectorizer.fit_transform(df_text)
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
feature_words = vectorizer.get_feature_names()
len(feature_words)

14190

In [11]:
df_words = pd.DataFrame(X.toarray(), columns=feature_words)
df_words.head()

,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,...,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df_filter = df_words.loc[:,(df_words.sum()>5)]
df_filter.shape

(7613, 2080)

In [13]:
tweets_final = pd.concat([tweets,df_filter], axis="columns")
tweets_final.shape

(7613, 2097)

In [14]:
X = tweets_final.drop(["id","keyword","location","text","target_label","hashtags","labels","clean_text"], axis=1)
y = tweets_final["target_label"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5709, 2087)
(1904, 2087)
(5709,)
(1904,)


In [17]:
#usando las 2087 features
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

LGBMClassifier()

In [19]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.779937


In [20]:
#vamos a reducir las 2087 features a las 600 mas importantes
model_lgb.feature_importances_

array([98, 12, 35, ...,  0,  0,  0])

In [21]:
df_feat_importances = pd.DataFrame(model_lgb.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [22]:
df_feat_importances.head(20)

,importancia
special_chars_count,98
text_length,97
vowels_count,81
short_words_count,62
stopwords_count,60
num_chars_count,52
words_count,45
labels_count,35
hiroshima,33
spill,29


In [23]:
list_fi = df_feat_importances.index[:600].tolist()
X = X.filter(items=list_fi)
X.shape

(7613, 600)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [49]:
#usando las 600 features
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

LGBMClassifier()

In [26]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.779937


#### Ajustando hiper-parametros:
    - n_estimators=100, num_leaves=7, colsample_bytree=0.3, subsample=1, max_depth=4, learning_rate=0.1
    SCORE: 0.758929
    - n_estimators=150, num_leaves=7, colsample_bytree=0.3, subsample=1, max_depth=4, learning_rate=0.1
    SCORE: 0.769958
    - n_estimators=200, num_leaves=7, colsample_bytree=0.3, subsample=1, max_depth=4, learning_rate=0.1
    SCORE: 0.777836
    - n_estimators=250, num_leaves=7, colsample_bytree=0.3, subsample=1, max_depth=4, learning_rate=0.1
    SCORE: 0.780462
    - n_estimators=300, num_leaves=7, colsample_bytree=0.3, subsample=1, max_depth=4, learning_rate=0.1
    SCORE: 0.778887
    - n_estimators=260, num_leaves=7, colsample_bytree=0.3, subsample=1, max_depth=4, learning_rate=0.1
    SCORE: 0.780987
#### Con Grid-Search 4-fold:
    - n_estimators=260, max_depth=7, colsample_bytree=0.3, learning_rate=0.1
    SCORE: 0.760899
    - n_estimators=260, max_depth=7, colsample_bytree=0.5, learning_rate=0.1
    SCORE: 0.763527
    - n_estimators=260, max_depth=7, colsample_bytree=0.7, learning_rate=0.1
    SCORE: 0.764403

In [43]:
model_lgb = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.3, subsample=1, max_depth=4, n_jobs=1,
                              n_estimators=260, min_child_samples=20, learning_rate=0.1)
model_lgb.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.3, max_depth=4, n_estimators=255, n_jobs=1,
               num_leaves=7, objective='binary', subsample=1)

In [44]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.780462


In [56]:
hiper_parametros = {"num_leaves":[7],
                   "max_depth":[7],
                    "n_estimators":[200],
                   "colsample_bytree":[0.3,0.5,0.7],
                   "learning_rate":[0.1,0.05,0.2],
                    "n_jobs": [1]}

In [57]:
clasif = GridSearchCV(model_lgb, hiper_parametros, cv=4, scoring='accuracy')
clasif.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=LGBMClassifier(),
             param_grid={'colsample_bytree': [0.3, 0.5, 0.7],
                         'learning_rate': [0.1, 0.05, 0.2], 'max_depth': [7],
                         'n_estimators': [200], 'n_jobs': [1],
                         'num_leaves': [7]},
             scoring='accuracy')

In [58]:
print(clasif.best_estimator_)
print(clasif.best_score_)

LGBMClassifier(colsample_bytree=0.7, learning_rate=0.2, max_depth=7,
               n_estimators=200, n_jobs=1, num_leaves=7)
0.7614258036781637


In [59]:
#ahora con tf-idf
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(tweets["clean_text"])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
df_words = pd.DataFrame(X.toarray(), columns=feature_words)
df_filter = df_words.loc[:,(df_words.sum()>2)]
df_filter.shape

(7613, 2072)

In [61]:
tweets_final = pd.concat([tweets,df_filter], axis="columns")
tweets_final.shape

(7613, 2089)

In [62]:
X = tweets_final.drop(["id","keyword","location","text","target_label","hashtags","labels","clean_text"], axis=1)
y = tweets_final["target_label"]

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [64]:
#usando las 2089 features
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

LGBMClassifier()

In [65]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.792542


In [66]:
df_feat_importances = pd.DataFrame(model_lgb.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [67]:
list_fi = df_feat_importances.index[:700].tolist()
X = X.filter(items=list_fi)
X.shape

(7613, 700)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [69]:
#usando 700 features
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

LGBMClassifier()

In [70]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.792542


In [101]:
model_lgb = lgb.LGBMClassifier(num_leaves=14, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=250, min_child_samples=10, learning_rate=0.15)
model_lgb.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.7, learning_rate=0.15, max_depth=7,
               min_child_samples=10, n_estimators=250, n_jobs=1, num_leaves=14,
               objective='binary', subsample=1)

In [102]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.786239


In [104]:
model_lgb.feature_importances_

array([168, 158, 161, 141, 125, 110,  35,  29,  18,  16,  23,  22, 103,
        36,  12,  16,  15,  18,  17,  19,  12,  14,  18,  17,  17,  14,
        12,  10,  15,  13,  18,  11,  13,  14,  12,  11,  13,  12,  11,
        11,  11,  11,   9,  13,  12,  10,   8,  11,  10,  15,  12,  14,
        12,  11,  11,   9,  10,   9,  10,   9,  11,   7,   6,  12,  12,
        11,   9,  10,   8,  12,   9,   9,   8,   9,  10,   9,   7,   6,
         6,   5,   7,  14,  12,   8,   6,   2,  11,   7,   4,   8,   4,
         6,   8,   5,   7,   8,   8,   3,   9,   5,   9,   4,   6,   5,
         7,   7,  10,   7,   5,   4,   5,   5,   6,  22,  11,   9,   9,
         7,   9,   6,   7,   5,   8,   8,   6,   4,   5,   6,  10,   8,
         8,   7,   7,   5,  11,   9,   9,   5,   9,   6,   4,   5,   5,
         7,   6,   4,   5,   4,   5,  10,   8,   5,   6,   8,   4,   7,
         5,   5,   6,   9,   2,   5,   2,   4,   7,   9,   6,   4,   3,
         7,   4,   4,   3,   3,   5,   5,   4,   5,   4,   5,   

In [105]:
df_feat_importances = pd.DataFrame(model_lgb.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [106]:
df_feat_importances.head(30)

,importancia
special_chars_count,168
vowels_count,161
text_length,158
stopwords_count,141
short_words_count,125
num_chars_count,110
words_count,103
labels_count,36
hiroshima,35
fires,29


In [108]:
df_feat_importances["importancia"].value_counts()

0      348
1       44
3       44
2       42
5       39
4       31
7       24
6       20
9       20
8       15
11      15
12      12
10      11
14       5
18       4
13       4
15       3
17       3
16       2
22       2
161      1
19       1
23       1
29       1
35       1
36       1
103      1
110      1
125      1
141      1
158      1
168      1
Name: importancia, dtype: int64

In [109]:
list_fi = df_feat_importances.index[:352].tolist()
X = X.filter(items=list_fi)
X.shape

(7613, 352)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [163]:
#usando 352 features
model_lgb = lgb.LGBMClassifier(num_leaves=14, objective="binary", colsample_bytree=0.6, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=300, min_child_samples=10, reg_lambda=1, learning_rate=0.2)
model_lgb.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.6, learning_rate=0.2, max_depth=7,
               min_child_samples=10, n_estimators=300, n_jobs=1, num_leaves=14,
               objective='binary', reg_lambda=1, subsample=1)

In [164]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.797269


In [165]:
model_lgb = lgb.LGBMClassifier(num_leaves=14, objective="binary", colsample_bytree=0.6, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=300, min_child_samples=10, reg_lambda=1, learning_rate=0.2)

In [170]:
kfold = KFold(n_splits=6, random_state=100)
resultados = cross_val_score(model_lgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 78.191889


In [171]:
kfold = StratifiedKFold(n_splits=6, random_state=100)
resultados = cross_val_score(model_lgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 78.016911
